In [19]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple

In [20]:
load_dotenv()

True

In [10]:
# ors_key = os.getenv("ORS_API_KEY")

# client = ors.Client(key= ors_key)

In [25]:
directory = os.getcwd()

os.chdir(directory)

priv_centroids_df = pd.read_csv(r"..\data\priv_cluster_centroids.csv",header = None,names = ['Latitude','Longitude'])
mrt_stations_df = pd.read_csv(r"..\data\mrt_station_final.csv",usecols = [1,2,3])

In [26]:
priv_centroids_df.info()
mrt_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   275 non-null    float64
 1   Longitude  275 non-null    float64
dtypes: float64(2)
memory usage: 4.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MRT.Name   175 non-null    object 
 1   Latitude   175 non-null    float64
 2   Longitude  175 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.2+ KB


In [27]:
mrt_stations_df.sort_values(by='MRT.Name', inplace=True)

mrt_stations_df.reset_index(inplace=True)

mrt_stations_df.head()

,index,MRT.Name,Latitude,Longitude
0,12,ADMIRALTY MRT STATION,1.440589,103.800990
1,9,ALJUNIED MRT STATION,1.316433,103.882906
2,52,ANG MO KIO MRT STATION,1.369429,103.849455
3,115,BAKAU LRT STATION,1.387994,103.905415
4,54,BANGKIT LRT STATION,1.380022,103.772647


In [28]:
#create a psuedo index for my residential centroids df
priv_centroids_df['index'] = priv_centroids_df.index

priv_centroids_df['index']

##create a dummy variable to cross join on 
priv_centroids_df['join_key'] = "A"
mrt_stations_df['join_key'] = "A"

#Cross join to obtain combinations of all possible pairings between MRTs and Residential Centroids
combined_df = pd.merge(priv_centroids_df, mrt_stations_df, on='join_key')

print(combined_df.head(1))

combined_df.info()

   Latitude_x  Longitude_x  index_x join_key  index_y               MRT.Name  \
0     1.36696   103.877984        0        A       12  ADMIRALTY MRT STATION   

   Latitude_y  Longitude_y  
0    1.440589    103.80099  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48125 entries, 0 to 48124
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Latitude_x   48125 non-null  float64
 1   Longitude_x  48125 non-null  float64
 2   index_x      48125 non-null  int64  
 3   join_key     48125 non-null  object 
 4   index_y      48125 non-null  int64  
 5   MRT.Name     48125 non-null  object 
 6   Latitude_y   48125 non-null  float64
 7   Longitude_y  48125 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 2.9+ MB


In [29]:
#Defining a function that calculates the Euclidean Distance between two points using Haversine Method?
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    distance = R * c
    return distance

In [30]:
coordinate_pair = namedtuple('coordinate_pair',['lat_x','lon_x','lat_y','lon_y'])

In [31]:
#Apply function to dataframe and store distances in new column 'euclidean distance'
combined_df['euclidean_distance'] = haversine(combined_df['Latitude_x'], combined_df['Longitude_x'], combined_df['Latitude_y'], combined_df['Longitude_y'])

result_df = combined_df.groupby('MRT.Name').apply(lambda group: group.nsmallest(5, 'euclidean_distance'),include_groups = False).reset_index(drop=True)

result_df['MRT_Name'] = pd.merge(result_df, mrt_stations_df, left_on='index_y', right_on='index')['MRT.Name']

result_df['coordinate_pair'] = result_df.apply(lambda x: coordinate_pair(x['Latitude_x'], x['Longitude_x'], x['Latitude_y'], x['Longitude_y']), axis=1)

# #Create a new column 'coordinate_pair' to store coordinate pairs to pass to openrouteservice API direction query
# result_df['coordinate_pair'] = list(zip(result_df['Longitude_x'], result_df['Latitude_x'], result_df['Longitude_y'], result_df['Latitude_y']))
# result_df['coordinate_pair'] = result_df['coordinate_pair'].apply(lambda x: [[x[0], x[1]], [x[2], x[3]]])


#create an empty column 'route' to later store query response
result_df['route'] = np.nan

result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Latitude_x          875 non-null    float64
 1   Longitude_x         875 non-null    float64
 2   index_x             875 non-null    int64  
 3   join_key            875 non-null    object 
 4   index_y             875 non-null    int64  
 5   Latitude_y          875 non-null    float64
 6   Longitude_y         875 non-null    float64
 7   euclidean_distance  875 non-null    float64
 8   MRT_Name            875 non-null    object 
 9   coordinate_pair     875 non-null    object 
 10  route               0 non-null      float64
dtypes: float64(6), int64(2), object(3)
memory usage: 75.3+ KB


In [10]:
# #Apply function to dataframe and store distances in new column 'euclidean distance'
# combined_df['euclidean_distance'] = haversine(combined_df['Latitude_x'], combined_df['Longitude_x'], combined_df['Latitude_y'], combined_df['Longitude_y'])

# #Group by residential centroid, filter out the closest MRT by distance for each centroid into another dataframe and reset its index
# result_df = combined_df.loc[combined_df.groupby('index_x')['euclidean_distance'].idxmin()].reset_index()

# result_df

# #Create a new column 'coordinate_pair' to store coordinate pairs to pass to openrouteservice API direction query
# result_df['coordinate_pair'] = list(zip(result_df['Longitude_x'], result_df['Latitude_x'], result_df['Longitude_y'], result_df['Latitude_y']))
# result_df['coordinate_pair'] = result_df['coordinate_pair'].apply(lambda x: [[x[0], x[1]], [x[2], x[3]]])


# #create an empty column 'route' to later store query response
# result_df['route'] = np.nan

In [33]:
one_map_email = os.getenv("ONE_MAP_EMAIL")
one_map_password = os.getenv("ONE_MAP_PASSWORD")
payload = {
        "email": one_map_email,
        "password": one_map_password
      }
api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
api_key = api_key.json()["access_token"]

In [34]:
onemap = OneMap(api_key)

In [35]:
def get_route(coordinate_pair):
    time.sleep(0.3)
    try:
        return onemap.routing.route(start_lat=coordinate_pair.lat_x, 
                                    start_lon=coordinate_pair.lon_x, 
                                    end_lat=coordinate_pair.lat_y, 
                                    end_lon=coordinate_pair.lon_y, 
                                    routeType="cycle")
    except Exception as e:
        print(f"Error: {e}")
        return None

In [36]:
result_df['route'] = result_df['coordinate_pair'].apply(get_route)

In [37]:
def get_distance(route):
    try:
        return route['route_summary']['total_distance']/1000 #convert m to km
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None

result_df['distance'] = result_df['route'].apply(get_distance)

def get_time(route):
    try:
        return route['route_summary']['total_time']/60 #convert second to minutes
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None
    

result_df['duration'] = result_df['route'].apply(get_time)

In [38]:
result_df

,Latitude_x,Longitude_x,index_x,join_key,index_y,Latitude_y,Longitude_y,euclidean_distance,MRT_Name,coordinate_pair,route,distance,duration
0,1.444223,103.807020,179,A,12,1.440589,103.800990,0.782691,ADMIRALTY MRT STATION,"(1.4442233028584144, 103.807020253402, 1.44058...",{'status_message': 'Found route between points...,0.830,4.983333
1,1.432023,103.799483,92,A,12,1.440589,103.800990,0.967067,ADMIRALTY MRT STATION,"(1.4320230193468555, 103.79948334428713, 1.440...",{'status_message': 'Found route between points...,5.012,30.533333
2,1.429926,103.784534,155,A,12,1.440589,103.800990,2.179939,ADMIRALTY MRT STATION,"(1.4299262711858671, 103.78453363789374, 1.440...",{'status_message': 'Found route between points...,2.711,16.800000
3,1.442710,103.824938,7,A,12,1.440589,103.800990,2.672379,ADMIRALTY MRT STATION,"(1.4427103141569004, 103.82493752346971, 1.440...",{'status_message': 'Found route between points...,4.003,25.583333
4,1.438786,103.826963,178,A,12,1.440589,103.800990,2.894081,ADMIRALTY MRT STATION,"(1.4387862182555462, 103.82696331794833, 1.440...",{'status_message': 'Found route between points...,4.286,27.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,1.431376,103.840189,215,A,78,1.429443,103.835005,0.615026,YISHUN MRT STATION,"(1.4313761188669467, 103.84018889548966, 1.429...",{'status_message': 'Found route between points...,1.016,7.116667
871,1.427091,103.825183,71,A,78,1.429443,103.835005,1.122700,YISHUN MRT STATION,"(1.427091447215155, 103.82518294734936, 1.4294...",{'status_message': 'Found route between points...,2.131,13.416667
872,1.438786,103.826963,178,A,78,1.429443,103.835005,1.370553,YISHUN MRT STATION,"(1.4387862182555462, 103.82696331794833, 1.429...",{'status_message': 'Found route between points...,2.403,14.950000
873,1.442710,103.824938,7,A,78,1.429443,103.835005,1.851687,YISHUN MRT STATION,"(1.4427103141569004, 103.82493752346971, 1.429...",{'status_message': 'Found route between points...,4.388,28.150000


In [39]:
def get_centroid_name(row):
    geocode = onemap.reverseGeocode.revGeoCode(row['Latitude_x'], row['Longitude_x'])
    if geocode['GeocodeInfo'][0]['BUILDINGNAME'] != "NIL":
        return geocode['GeocodeInfo'][0]['BUILDINGNAME']
    else:
        if geocode['GeocodeInfo'][0]['BLOCK'] != "NIL":
            return geocode['GeocodeInfo'][0]['BLOCK'] + " " + geocode['GeocodeInfo'][0]['ROAD']
        else:
            return geocode['GeocodeInfo'][0]['ROAD']

In [40]:
result_df['centroid_name'] = result_df.apply(get_centroid_name, axis=1)

In [41]:
new_result_df = result_df. loc[:, result_df. columns != 'route']
new_result_df.to_csv(r"..\data\Private_Centroid_MRT pairing data_5_cluster.csv")
routes = result_df['route'].copy(deep = True)
routes.to_json(r'..\data\Private_MRT_routes_5_cluster.json', orient='records')

In [42]:
result_df[result_df['route'].isna()]

,Latitude_x,Longitude_x,index_x,join_key,index_y,Latitude_y,Longitude_y,euclidean_distance,MRT_Name,coordinate_pair,route,distance,duration,centroid_name


In [43]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Latitude_x          875 non-null    float64
 1   Longitude_x         875 non-null    float64
 2   index_x             875 non-null    int64  
 3   join_key            875 non-null    object 
 4   index_y             875 non-null    int64  
 5   Latitude_y          875 non-null    float64
 6   Longitude_y         875 non-null    float64
 7   euclidean_distance  875 non-null    float64
 8   MRT_Name            875 non-null    object 
 9   coordinate_pair     875 non-null    object 
 10  route               875 non-null    object 
 11  distance            875 non-null    float64
 12  duration            875 non-null    float64
 13  centroid_name       875 non-null    object 
dtypes: float64(7), int64(2), object(5)
memory usage: 95.8+ KB
